In [ ]:
# Boilerplate code from https://gist.github.com/fchollet/0830affa1f7f19fd47b06d4cf89ed44d
# Documentation: https://keras.io/preprocessing/image/
# Adapted by Kris Buote for Avalanche prediction

# The Keras image generator takes existing images and uses the subdirectories as classes. 
# It creates a larger data set by doing real-time data augmentation.

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import os, os.path


# dimensions of our images.
img_width, img_height = 432, 288

train_data_dir = '../data/temp_prec_images/Below Treeline/train/'
validation_data_dir = '../data/temp_prec_images/Below Treeline/validation'

nb_train_samples = sum([len(files) for r, d, files in os.walk(train_data_dir)]) # count the number of training images
nb_validation_samples = sum([len(files) for r, d, files in os.walk(validation_data_dir)])

epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5)) # 5 neuron outputs danger rating prediction vector. i.e. [0, 0, 1, 0, 0] means rating 3
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False) # consider horizontal_flip = True. I think it would just be "mirrored in time"

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    shuffle=True,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    shuffle=True,
    class_mode='categorical')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')



Found 285 images belonging to 5 classes.
Found 62 images belonging to 5 classes.
Epoch 1/50
17/17 [==============================] - 78s 5s/step - loss: 2.3265 - acc: 0.3897 - val_loss: 1.2883 - val_acc: 0.3125
Epoch 2/50
17/17 [==============================] - 75s 4s/step - loss: 1.2438 - acc: 0.3748 - val_loss: 1.0313 - val_acc: 0.5217
Epoch 3/50
17/17 [==============================] - 88s 5s/step - loss: 1.2310 - acc: 0.4471 - val_loss: 1.1498 - val_acc: 0.2609
Epoch 4/50
17/17 [==============================] - 100s 6s/step - loss: 1.2619 - acc: 0.4115 - val_loss: 1.0473 - val_acc: 0.5217
Epoch 5/50
17/17 [==============================] - 104s 6s/step - loss: 1.2331 - acc: 0.3740 - val_loss: 1.1499 - val_acc: 0.5000
Epoch 6/50
17/17 [==============================] - 113s 7s/step - loss: 1.2047 - acc: 0.3858 - val_loss: 1.0505 - val_acc: 0.5217
Epoch 7/50
15/17 [=========================>....] - ETA: 9s - loss: 1.2127 - acc: 0.4333 